# **Facial Expression Recognition**

In [2]:
# Libraries

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input,Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization,Dense
from keras.preprocessing.image import ImageDataGenerator as Imgen
from keras.callbacks import EarlyStopping,ModelCheckpoint

from sklearn.metrics import confusion_matrix,classification_report

## **Reading and Organiing Data**

In [3]:
# reading the data
data = pd.read_csv("../input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv")
data.head()


In [4]:
data

In [5]:
pixel_data = data[' pixels']
label_data = data['emotion']

In [6]:
len(label_data)

In [7]:
def preprocess_pixels(pixel_data):
  images = []
  for i in range(len(pixel_data)):
    img = np.fromstring(pixel_data[i], dtype='int', sep=' ')
    img = img.reshape(48,48,1)
    images.append(img)

  X = np.array(images)

 

  return X

In [8]:
oversampler = RandomOverSampler(sampling_strategy='auto')

X_over, Y_over = oversampler.fit_resample(pixel_data.values.reshape(-1,1), label_data)

In [9]:
X_over_series = pd.Series(X_over.flatten())
X_over_series

### **Visualaizing some Images**

In [10]:
X = preprocess_pixels(X_over_series)
Y = Y_over

In [11]:
import pandas as pd
#from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Input, BatchNormalization, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras.models import Model
from keras.utils import to_categorical
from sklearn.metrics import classification_report
#from imblearn.over_sampling import RandomOverSampler
from keras.preprocessing import image
import scipy
import os
import cv2

In [12]:
Y = Y_over.values.reshape(Y.shape[0],1)
Y.shape

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 45)

In [14]:
def emotion_recognition(input_shape):

  X_input = Input(input_shape)

  X = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid')(X_input)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)


  X = Conv2D(64, (3,3), strides=(1,1), padding = 'same')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)

  X = MaxPooling2D((2,2))(X)

  X = Conv2D(64, (3,3), strides=(1,1), padding = 'valid')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)

  X = Conv2D(128, (3,3), strides=(1,1), padding = 'same')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)


  X = MaxPooling2D((2,2))(X)

  X = Conv2D(128, (3,3), strides=(1,1), padding = 'valid')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)

 

  X = MaxPooling2D((2,2))(X)
  X = Flatten()(X)
  X = Dense(200, activation='relu')(X)
  X = Dropout(0.6)(X)
  X = Dense(7, activation = 'softmax')(X)

  model = Model(inputs=X_input, outputs=X)

  return model

In [15]:
model = emotion_recognition((48,48,1))

In [16]:
model.summary()

In [17]:
adam = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
y_train = to_categorical(Y_train, num_classes=7)

In [19]:
y_train.shape

In [20]:
y_test = to_categorical(Y_test, num_classes=7)
model.fit(X_train, y_train, epochs = 100, validation_data=(X_test, y_test))

In [21]:
model.save_weights('emotion_weights_3.hdf5')

In [22]:
import tensorflow as tf
from keras.models import load_model

model.save('my_model.h5')

In [23]:
model.evaluate(X_test, y_test)

In [24]:
preds = model.predict(X_train)

In [25]:
def get_class(preds):
  pred_class = np.zeros((preds.shape[0],1))

  for i in range(len(preds)):
   pred_class[i] = np.argmax(preds[i])

  return pred_class

In [26]:
pred_class_train = get_class(preds)

In [27]:
train_report = classification_report(Y_train, pred_class_train)

In [28]:
print(train_report)

In [29]:
test_preds = model.predict(X_test)
len(test_preds)

In [30]:
pred_test_class = get_class(test_preds)

In [31]:
report_test = classification_report(Y_test, pred_test_class)

In [32]:
print(report_test)

In [33]:
label_dict = {0 : 'Angry', 1 : 'Disgust', 2 : 'Fear', 3 : 'Happiness', 4 : 'Sad', 5 : 'Surprise', 6 : 'Neutral'}
label_dict

[](http://)

In [43]:
img_path = "../input/image11/IMG_20211202_144326.jpg"
img = image.load_img(img_path, grayscale=True, target_size=(48,48))
img = image.load_img(img_path, grayscale=True, target_size=(48,48))

In [44]:
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x.shape

In [45]:
prediction = np.argmax(model.predict(x))
print('The predicted emotion is : ' + label_dict[prediction])
my_image = image.load_img(img_path)
plt.imshow(my_image)

In [37]:
#myimgg = Image(filename="../input/my-test-image-set/person.png", width= 48, height=48)
